In [1]:
using TensorDec
using DynamicPolynomials
using MomentTools
using CSDP, JuMP
# The function "Optimizer" is a global optimization solver based on positive semi-definite programming
optimizer = CSDP.Optimizer
using LinearAlgebra

# Example 1:

In [2]:
# Define the parameters
X = @polyvar x1 x2 x3

(x1, x2, x3)

In [3]:
# P is a homogeneous polynomial of degree 4 in 3 variables
P = (x1+x2+0.75*x3)^4+1.5*(x1-x2)^4-2*(x1-x3)^4;

The graph of P in polar coordinates on the sphere looks like this:

![title](pol_img.png)

Let us use the function "optimizer" to minimize and maximize P on the unit sphere. The maximum evaluation of P <b>in absolute value</b> on the unit sphere (and that is why we have to use both maximize and minimize functions) gives the spectral norm of P and equivalently a best rank-1 approximation of the symmetric tensor associated to P.

In [4]:
v1, M1 = minimize(P, [x1^2+x2^2+x3^2-1], [], X, 8, optimizer);
v2, M2 = maximize(P, [x1^2+x2^2+x3^2-1], [], X, 8, optimizer);

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 6.27e-01 Pobj: -9.1377312e+00 Ad: 6.65e-01 Dobj:  6.5822601e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -8.9378911e+01 Ad: 5.42e-01 Dobj:  6.7430765e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -7.8427861e+01 Ad: 8.83e-01 Dobj:  1.1629773e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -4.7706795e+01 Ad: 8.97e-01 Dobj:  3.8603631e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.8751219e+01 Ad: 7.72e-01 Dobj:  5.1044080e-02 
Iter:  6 Ap: 7.86e-01 Pobj: -8.9568745e+00 Ad: 9.24e-01 Dobj: -1.6229630e+00 
Iter:  7 Ap: 9.83e-01 Pobj: -8.6192083e+00 Ad: 7.44e-01 Dobj: -6.4997642e+00 
Iter:  8 Ap: 7.68e-01 Pobj: -7.8344478e+00 Ad: 7.62e-01 Dobj: -7.2868769e+00 
Iter:  9 Ap: 9.65e-01 Pobj: -7.7124187e+00 Ad: 8.41e-01 Dobj: -7.6028422e+00 
Iter: 10 Ap: 9.33e-01 Pobj: -7.7024600e+00 Ad: 9.51e-01 Dobj: -7.6931385e+00 
Iter: 11 Ap: 9.37e-01 Pobj: -7.7016436e+00 Ad: 9.79e-01 Dobj: -7.7012173e+00 
Iter: 12 Ap: 9.65e-01 Pobj: -7.7015833e+00 Ad: 1.00e+

The minimum evaluation of P on the unit sphere is: -7.701579459519532.

The maximum evaluation of P on the unit sphere is: 6.565249952183416.

Thus, the maximum weight in absolute value which is the spectral norm of P is: 7.701579459519532.

The unit vectors that give this value are: [0.6805571886747267, 0.048429787707634814, -0.7310926539221407] and [-0.6805571886747267, -0.048429787707634814, 0.7310926539221407].

Let us compute a rank-1 approximation of P. We will compute an initial point by the method SMD, the Julia function that corresponds to this method in the package TensorDec is called "decompose", then we will use the Riemannian Newton algorithm with trust region scheme for the real case, the corresponding Julia function in the package TensorDec is called "rne_n_tr_r".

In [5]:
# Compute an initial point
w1, V1 = decompose(P,1)

([-7.325574933843033], [-0.6639023290661047; -0.03627133467952941; 0.7469391459424036;;], Dict{String, Any}("diagonalization" => Dict{String, Any}("case" => "1x1")))

We can notice that the weight given by decompose is very close to the one given by Optimizer. Let us refine this point by using a few number of iterations of rne_n_tr_r, for example 5 iterations.

In [6]:
w_end, V_end, Info = approximate(P, w1, V1; iter= :RNE)

([-7.701578959506499], [-0.6805321710696051; -0.048475557058647804; 0.7311129081798128;;], Dict{String, Any}("d*" => 8.819735873970322, "d0" => 8.836665566369318, "nIter" => 5, "epsIter" => 0.001, "maxIter" => 5))

The weight in absolute value given by rne_n_tr_r initialized by decompose for rank-1 symmetric tensor approximation is: 7.701576525649196.

The unit vector given by rne_n_tr_r initialized by decompose for rank-1 symmetric tensor approximation is: [0.68061769889553; 0.04831896554028091; -0.7310436550024019].

Verifying with the global optimization method "optimizer" from the package CSDP, the symmetric rank-1 approximation $w_{end}(v_{end}^tX)^4$ of P given by "rne_n_tr_r" initialized by "decompose" is a best rank-1 approximation.

# Example 2:

Let us take a random symmetric tensor normally distributed with complex coefficients of order 4 and dimension 3 (the generic symmetric rank is 5), and let us compute by the Riemannian Newton algorithm "rne_n_tr" and the Riemannian Gauss--Newton algorithm "rgn_v_tr" initialized by a random initial point obeying normal distribution an approximated rank-3 symmetric tensor.   

In [7]:
# Take a random symmetric tensor
using Tensors
n = 3; d = 4; r = 3
T = randn(SymmetricTensor{d, n})+randn(SymmetricTensor{d, n})*im
T = convert(Array,T)
# show the first 3 arrays of T
T[:,:,:,1]

3×3×3 Array{ComplexF64, 3}:
[:, :, 1] =
   1.05942+0.967406im    -0.192566-1.16394im      0.922106+1.00663im
 -0.192566-1.16394im     -0.344832+1.53494im   -0.00689795-0.536803im
  0.922106+1.00663im   -0.00689795-0.536803im   -0.0279183-0.208539im

[:, :, 2] =
    1.1632-0.738255im  -0.304006-1.88664im    0.444836+0.293018im
 -0.304006-1.88664im     1.01849+1.39391im   -0.508148+0.311036im
  0.444836+0.293018im  -0.508148+0.311036im    1.99717-1.09032im

[:, :, 3] =
   1.06936+1.25084im  -1.13379-0.07772im   -0.238514-2.1973im
  -1.13379-0.07772im   1.54039+0.98697im    0.161779+0.388619im
 -0.238514-2.1973im   0.161779+0.388619im    1.98842+1.61457im

In [8]:
# Take the associate homogeneous polynomial P to T by applying the function ahp (for associate homogeneous polynomial)
X = (@polyvar x[1:n])[1]
P = ahp(T, X);

In [9]:
# Take an initial point 
w = ones(r) + fill(0.0+0.0im,r);
V = randn(ComplexF64,n,r);

# Apply Riemannian Newton Exact (rne_n_tr) iterations:
#w_end, V_end, Info = rne_n_tr(P, w, V, Dict{String,Any}("maxIter" => 500, "epsIter" => 1.e-3))
w_end, V_end, InfoRNE = approximate(P, w, V; iter = :RNE)

([1.1096671719861202e-9, 5.98212387773361, 2.682113097874022e-19], ComplexF64[0.2987112923750196 - 0.13909086644108615im -0.7821800221508292 - 0.15219718844554106im 0.011982519347201846 + 0.5836934646937137im; -0.2614740158925517 - 0.09217093656421778im -0.14104997599851551 + 0.22379290517635567im 0.33612574364148207 + 0.06560131138994613im; -0.5933344320153391 - 0.6800848505379644im -0.5023317716876389 - 0.2066757358620108im 0.18522057136760825 - 0.7124378224459865im], Dict{String, Real}("d*" => 8.846369749422273, "d0" => 28.872810525194193, "nIter" => 10, "epsIter" => 0.001, "maxIter" => 500))

The reported error is the apolar norm between P and the approximated polynomial. 

The initial error is:

In [10]:
InfoRNE["d0"]

28.872810525194193

The final error is:

In [11]:
InfoRNE["d*"]

8.846369749422273

The number of iterations is:

In [12]:
InfoRNE["nIter"]

10

In [13]:
# Adjust the initial point to use't with rgn_v_tr since this function take only the matrix V as parameters without the weight vector
#for i in 1:r
#    V[:,i]=(w[i])^(1/d)*V[:,i]
#end

# Applying Riemannian Gauss Newton (rgn_v_tr) iterations
#V_end, Info = rgn_v_tr(P, V, Dict{String,Any}("maxIter" => 500,"epsIter" => 1.e-3))
w_end, V_end, InfoRGN = approximate(P, w, V; iter = :RGN)


(ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im], ComplexF64[-0.5881594173179924 - 0.025385385467936915im -1.2187274080293686 - 0.24839351865999887im 0.642909052916074 - 0.5454490823918139im; 1.2166965793927451 + 0.06884746914437198im -0.20628778612331322 + 0.35469995154084355im 0.8178210730230298 - 0.2616942929596881im; 0.48811465424505024 - 0.4426753765165494im -0.7926639551975484 - 0.31704860839996474im -0.7064689178737351 + 0.3678276568239852im], Dict{String, Real}("d*" => 5.753217330627032, "d0" => 28.872810525194193, "nIter" => 44, "epsIter" => 0.001, "maxIter" => 500))

The reported error is the apolar norm between P and the approximated polynomial.

The initial error is:

In [14]:
InfoRGN["d0"]

28.872810525194193

The algorithm takes 10 iterations and decreases to the final error d*:

In [15]:
InfoRGN["d*"]

5.753217330627032

The number of iterations is:

In [16]:
InfoRGN["nIter"]

44